# Analysis of words in summaries to determine character actions [Work in progress]

The list of all the abbreviations https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
The verbs are given by :
- VB
- VBD
- VBG
- VBN
- VBP
- VBZ

The adjectives are given by :

- JJ
- JJR
- JJS

The nouns are given by:
- NN
- NNS
- NNP
- NNPS

## Pipeline to extract verbs, adjectives and nouns from the plot summary

As stopwords appear very frequently in the text they mask the important words. We therefore remove them from the text.

# Once this function is finalized move it to helpers.py file (and comment it correctly)

The outputs (pickle and csv) can be downloaded at https://drive.switch.ch/index.php/s/U5ntGAtMkXWi9yn. They are not pushed onto the repository as they are too large (>100MB)

## Assigning genders to characters

The number of missing genders is reasonable, it is therefore not necessary to predict them for the rest of the analysis. If it later comes out that this is required packages such as gender-guesser (https://pypi.org/project/gender-guesser/) can be used

The number of missing character's name is close to 25% so we will be able to perform our analysis only on 75% of the characters in our analysis.

## From the summary, extract the characters that appear and matches them with the character_data and associate the gender of the actor

## Now "scans" each summary for each character (perfect match or not) and associate them the previous and folllowing 3 words in the summmary, these words will be analyzed later on).

## Last step, take the words extracted at the previous step and classify them into verbs nouns and adjectives, and depending on the actor's gender, append them to the designated dictionnary

In [ ]:
# Load the datasets
characters_data=pd.read_csv('DATA/characters_data.csv',low_memory=False)
movie_data=pd.read_csv('DATA/movie_data.csv',low_memory=False)

In [ ]:
# Count number of nans in gender column
missing_gender_percentage = 100 * characters_data['actor_gender'].isna().sum() / len(characters_data.index)
print(f"Number of missing genders: {missing_gender_percentage:.2f}%")
# Count number of nans in character name column
missing_names_percentage = 100 * characters_data['character_name'].isna().sum() / len(characters_data.index)
print(f"Number of missing character names: {missing_names_percentage:.2f}%")

In [ ]:
# drop the characters with missing character names and actors gender so we can determine if characters are F/M
characters_noNA = characters_data.dropna(subset=["character_name", "actor_gender"])
# drop movies with missing summaries
movie_noNA = movie_data.dropna(subset=["plot_summary"])

In [191]:
# List of movies that we will use in the analysis - must have summary, character name and gender
filtered_df = movie_noNA[movie_noNA['IMDB_ID'].isin(characters_noNA['IMDB_ID'])]
filtered_df

,IMDB_ID,wikipedia_ID,freebase_ID,box_office_revenue,runtime,name,release_date,genre,plot_summary
0,tt0228333,975900.0,/m/03vyhn,14010832.0,98.0,Ghosts of Mars,2001.0,"['Thriller', 'Adventure', 'Horror', 'Action', ...","Set in the second half of the 22nd century, th..."
5,tt0119548,6631279.0,/m/0gffwj,NaN,93.0,Little city,1997.0,"['Comedy', 'Romance Film', 'Drama', 'Romantic ...","Adam, a San Francisco-based artist who works a..."
6,tt0058331,77856.0,/m/0kcn7,102272727.0,139.0,Mary Poppins,1964.0,"['Musical', ""Children's/Family"", 'Drama', 'Com...",The film opens with Mary Poppins perched in a...
10,tt0892904,21926710.0,/m/05p45cv,NaN,82.0,White on Rice,2009.0,[],"[""Forty-year-old Jimmy is growing up--or at le..."
12,tt0255819,156558.0,/m/014k4y,29381649.0,123.0,Baby Boy,2001.0,"['Drama', 'Coming of age']",A young 20-year-old named Jody lives with his...
...,...,...,...,...,...,...,...,...,...
53602,tt9908448,NaN,NaN,NaN,NaN,The Bells of Hell,2018.0,[],['An alcoholic newlywed wife spends three days...
53603,tt9908592,NaN,NaN,NaN,NaN,Filmmakers Unite (FU),2017.0,[],"['Curated by Jay Rosenblatt and Ellen Bruno, F..."
53605,tt9910648,NaN,NaN,NaN,NaN,"The Good Americans: One Revolution, Two Nations",2021.0,[],['Størmerlige Films is making a feature-length...
53608,tt9913288,NaN,NaN,NaN,NaN,Letters,2020.0,[],"[""Charlie Porter is an ordinary man. He leads ..."


In [146]:
#output_characters_genders=find_characters_genders_for_all_movies(filtered_df, characters_noNA)

In [147]:
# Save output to csv
output_characters_genders.to_csv('DATA/output_characters_genders.csv', index=False)

NameError: name 'output_characters_genders' is not defined

In [ ]:
#Export as pickle
output_characters_genders.to_pickle('pickles/output_characters_genders.pickle')

In [ ]:
output_characters_genders

In [ ]:
count_gender = output_characters_genders['gender'].value_counts().to_dict()

print(
    "There are only {} movies left and with {} male characters and {} female characters.".format(
        len(output_characters_genders['IMDB_ID'].unique()), count_gender.get("M", 0), count_gender.get("F", 0)
    )
)

In [ ]:
#output_characters_genders_context=extract_context_strings(output_characters_genders)

In [ ]:
# Save result as csv
output_characters_genders_context.to_csv('DATA/output_characters_genders_context.csv', index=False)

In [ ]:
# Save result as pickle
output_characters_genders_context.to_pickle('pickles/output_characters_genders_context.pickle')

In [44]:
# Load from csv
output_characters_genders_context=pd.read_csv('DATA/output_characters_genders_context.csv')

In [45]:
output_characters_genders_context

,IMDB_ID,plot_summary,character_name,gender,associated_words
0,tt0228333,"Set second half 22nd century, film depicts Mar...",Sgt Jericho Butler,M,second half 22nd humans walk surface wearing ...
1,tt0228333,"Set second half 22nd century, film depicts Mar...",Bashira Kincaid,F,pick transport prisoner Desolation Williams A...
2,tt0228333,"Set second half 22nd century, film depicts Mar...",Michael Descanso,M,Set second 22nd century film 22nd century fil...
3,tt0228333,"Set second half 22nd century, film depicts Mar...",Big Daddy Mars,M,century film depicts planet 84 terraformed
4,tt0228333,"Set second half 22nd century, film depicts Mar...",Akooshay,F,Set second 22nd century film miners discovere...
...,...,...,...,...,...
176329,tt9914522,"[""When Mackenzie Holden's sister brother law m...",Katelyn,F,NaN
176330,tt9914522,"[""When Mackenzie Holden's sister brother law m...",Kelsey,F,NaN
176331,tt9914522,"[""When Mackenzie Holden's sister brother law m...",Jade,F,honeymoon Mackenzie new Evan decision make ma...
176332,tt9914522,"[""When Mackenzie Holden's sister brother law m...",Evan,M,Mackenzie new husband decision make regarding


In [54]:
verbs, adjs, nouns, chunks = extract_words(output_characters_genders_context, "IMDB_ID", "character_name","associated_words")

Processing Movies: 100%|██████████| 176334/176334 [18:35<00:00, 158.04it/s] 


In [64]:
# Create DataFrames from the lists
verbs_df = pd.DataFrame(verbs, columns=['IMDB_ID', 'character_name', 'Verbs'])
adjs_df = pd.DataFrame(adjs, columns=['IMDB_ID', 'character_name', 'Adjectives'])
nouns_df = pd.DataFrame(nouns, columns=['IMDB_ID', 'character_name', 'Nouns'])
chunks_df = pd.DataFrame(chunks, columns=['IMDB_ID', 'character_name', 'Chunks'])

In [56]:
# Merge DataFrames on 'IMDB_ID' and 'character_name'
final_df = verbs_df.merge(adjs_df, on=['IMDB_ID', 'character_name']) \
                    .merge(nouns_df, on=['IMDB_ID', 'character_name']) \
                    .merge(chunks_df, on=['IMDB_ID', 'character_name'])

In [58]:
final_df

,character_name,Verbs,Adjectives,Nouns,Chunks
0,Sgt Jericho Butler,"[walk, wearing, sent, opened, released, posses...","[second, pick, disembodied, possible, Unfortun...","[half, humans, surface, pressure, suits, team,...","[(second, JJ), (half, NN), (22nd, CD), (humans..."
1,Bashira Kincaid,"[killed, returning, blame, cot, escapes, leaving]",[massacre],"[pick, transport, prisoner, Desolation, Willia...","[(pick, NN), (transport, NN), (prisoner, NN), ..."
2,Michael Descanso,"[planet, finds, missing, discovered, discovere...","[second, 22nd, 22nd, doorway, ancient, Martian...","[Set, century, film, century, film, Mars, mini...","[[(Set, NNP)], (second, JJ), (22nd, JJ), (cent..."
3,Big Daddy Mars,"[planet, terraformed]",[],"[century, film, depicts]","[(century, NN), (film, NN), (depicts, NNS), (p..."
4,Akooshay,"[discovered, created, wiped]","[second, 22nd, ancient, fierce]","[Set, century, film, miners, Martian, miners, ...","[[(Set, NNP)], (second, JJ), (22nd, JJ), (cent..."
...,...,...,...,...,...
45040982,Jewel,[Rose],[mysterious],"[letter, stalker, catalepsy, condition, condit...","[(found, VBN), (drug, NN), (attic, JJ), (prost..."
45040983,Jewel,[Rose],[mysterious],"[letter, stalker, catalepsy, condition, condit...","[(young, JJ), (couple, NN), [(Nathan, NNP)], (..."
45040984,Jewel,[Rose],[mysterious],"[letter, stalker, catalepsy, condition, condit...","[(part, NN), (true, JJ), (story, NN), (gospel,..."
45040985,Jewel,[Rose],[mysterious],"[letter, stalker, catalepsy, condition, condit...","[(father, RB), (introduced, VBN), (intriguing,..."


In [ ]:
# to have the genders, use that df if it is too big to merge with the character_data and save to csv
# DO NOT RUN THAT CELL IF RUN THE PREVIOUS ONE DOES BECAUSE THE GENDER COLUMN WILL BE DUPLICATED
# max l'a run lol
final_df = pd.merge(final_df, output_characters_genders_context[['IMDB_ID', 'character_name', 'gender']], on=['IMDB_ID', 'character_name'], how='left')

In [59]:
characters_data=pd.read_csv('DATA/characters_data.csv',low_memory=False)

KeyboardInterrupt: 

In [ ]:
to_save_df = pd.merge(characters_data, final_df, on=['IMDB_ID', 'character_name'], how='left')

In [ ]:
to_save_df

In [ ]:
to_save_df.to_csv('DATA/characters_personas_data.csv', index=False)

In [ ]:
male_dict, female_dict = create_gender_dictionaries(result_df_with_context)
print("Male Dictionary:")
print(male_dict)
print("\nFemale Dictionary:")
print(female_dict)

# Calculate word frequencies for male and female characters
male_frequencies = calculate_word_frequencies(male_dict)
female_frequencies = calculate_word_frequencies(female_dict)

# Print or use the frequencies as needed
print("Male Word Frequencies:")
print(male_frequencies)
print("\nFemale Word Frequencies:")
print(female_frequencies)

In [ ]:
import matplotlib.pyplot as plt

def plot_top_words(frequencies, category, gender, top_n=5):
    words = list(frequencies[category].keys())
    values = list(frequencies[category].values())

    # Get the top N words and their frequencies
    top_words = [word for _, word in sorted(zip(values, words), reverse=True)[:top_n]]
    top_frequencies = [frequencies[category][word] for word in top_words]

    # Plot the bar chart
    plt.figure(figsize=(10, 6))
    plt.bar(top_words, top_frequencies, color='skyblue')
    plt.title(f'Top 5 {category} Words for {gender} Characters')
    plt.xlabel('Words')
    plt.ylabel('Frequency')
    plt.show()

# Example usage:
# Assuming you have already obtained the male and female frequencies
# male_frequencies, female_frequencies = calculate_word_frequencies(male_dict), calculate_word_frequencies(female_dict)

# Plot the top 5 words for each category for male characters
for category in ['Verbs', 'Adjectives', 'Nouns']:
    plot_top_words(male_frequencies, category, 'Male')

# Plot the top 5 words for each category for female characters
for category in ['Verbs', 'Adjectives', 'Nouns']:
    plot_top_words(female_frequencies, category, 'Female')

